In [ ]:
import kagglehub
import os
import shutil
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets, models, transforms
import torch.optim as optim
from torchvision.models import resnet50, ResNet50_Weights
from tqdm import tqdm
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
import shutil
import getpass
import nbformat
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
#Download latest version of the dataset
path = kagglehub.dataset_download('obulisainaren/multi-cancer')

print('Path to dataset files:', path)

In [ ]:
#Path check
dataset_path = os.path.join(path, 'Multi Cancer', 'Multi Cancer')
print('Using dataset_path:', dataset_path)
print(os.listdir(dataset_path))


In [ ]:
#Download ResNet50 model
model = resnet50(weights=ResNet50_Weights.DEFAULT)

In [ ]:

#Paths and device check
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print('GPU:', torch.cuda.get_device_name(0))
    print('Total GPU memory:', round(torch.cuda.get_device_properties(0).total_memory / 1e9, 2), 'GB')


transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


# Collect all subfolders as classes
all_subfolders = []
for main_class in os.listdir(dataset_path):
    main_path = os.path.join(dataset_path, main_class)
    if os.path.isdir(main_path):
        for sub_class in os.listdir(main_path):
            sub_path = os.path.join(main_path, sub_class)
            if os.path.isdir(sub_path):
                all_subfolders.append(sub_path)

# ap subfolder paths to class indices and create samples list
samples = []
class_to_idx = {}
for idx, subfolder_path in enumerate(all_subfolders):
    class_name = os.path.basename(subfolder_path)
    class_to_idx[class_name] = idx
    for fname in os.listdir(subfolder_path):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            samples.append((os.path.join(subfolder_path, fname), idx))

print('Detected classes:', list(class_to_idx.keys()))
print('Total images:', len(samples))


#Dataset to deal with nested folders
class NestedImageDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

full_dataset = NestedImageDataset(samples, transform=transform_train)

#Train/Test/Val Split
total_size = len(full_dataset)
test_size = int(0.1 * total_size)
val_size = int(0.1 * total_size)
train_size = total_size - val_size - test_size

generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size], generator=generator
)

#Evaluation transforms
val_dataset.dataset.transform = transform_eval
test_dataset.dataset.transform = transform_eval

print(f'Train size: {len(train_dataset)}, Val size: {len(val_dataset)}, Test size: {len(test_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)


model = resnet50(weights=ResNet50_Weights.DEFAULT)        #Model
num_classes = len(class_to_idx)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scaler = torch.cuda.amp.GradScaler()


#Training
epochs = 25
best_val_acc = 0.0
checkpoint_dir = '/content/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Training]', leave=False)

    for images, labels in pbar:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        pbar.set_postfix({'Batch Loss': loss.item(),
                          'GPU Mem (GB)': round(torch.cuda.memory_allocated(0)/1e9, 2)})


    #Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    print(f'\nEpoch {epoch+1}/{epochs} Summary:')
    print(f'  Train Loss: {total_loss/len(train_loader):.4f}')
    print(f'  Val Loss:   {val_loss/len(val_loader):.4f}')
    print(f'  Val Acc:    {val_acc:.2f}%')

    #Saves checkpoints if they improve
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        checkpoint_path = os.path.join(checkpoint_dir, f'best_model_epoch{epoch+1}.pt')
        torch.save(model.state_dict(), checkpoint_path)
        print(f'  New best: {checkpoint_path}')

    if device.type == 'cuda':
        print(f'  GPU Memory Allocated: {round(torch.cuda.memory_allocated(0)/1e9,2)} GB')
        print(f'  GPU Memory Cached:    {round(torch.cuda.memory_reserved(0)/1e9,2)} GB')
    print('='*50)

print('Training complete.')

#Test Evaluation
print('\nEvaluating on full test set...')
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc='Testing'):
        images, labels = images.to(device), labels.to(device)
        with torch.cuda.amp.autocast():
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

# Per-class metrics
print('\nPer-class metrics:')
print(classification_report(all_labels, all_preds, target_names=list(class_to_idx.keys())))

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print('\nConfusion Matrix:\n', cm)

In [ ]:
#Model Initialization (For if you'd like to try this out yourself the model best_model_epoch5.pt in the repo)
#Only run this code if you DON'T want to train your own model and would rather use the pretrained model

#Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Model Initialization
num_classes = 26  # number of classes in your trained model
model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # number of classes
model = model.to(device)

#Load trained weights
BEST_MODEL = "/content/checkpoints/best_model_epoch5.pt"  # Download the model from the repo and paste the path here
model.load_state_dict(torch.load(BEST_MODEL, map_location=device))
model.eval()

In [ ]:
model.eval()  # ensure evaluation mode

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        with torch.cuda.amp.autocast():
            outputs = model(images)
            _, predicted = outputs.max(1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Overall accuracy
overall_acc = accuracy_score(all_labels, all_preds)
print(f'Overall Accuracy: {overall_acc*100:.2f}%\n')

# Per-class precision, recall, F1-score
print('Per-class metrics:\n')
target_names = list(class_to_idx.keys())  # ensure correct class names
print(classification_report(
    all_labels, all_preds,
    target_names=target_names,
    digits=4  # show 4 decimal places
))

In [ ]:
#Ensure the model is in evaluation mode
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        with torch.cuda.amp.autocast():
            outputs = model(images)
            _, predicted = outputs.max(1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # normalize

# Plot matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=list(class_to_idx.keys()),
            yticklabels=list(class_to_idx.keys()))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Normalized Confusion Matrix')
plt.show()

In [29]:
def classify_random_images(subset, model, device, num_images=20, transform=None):    #Randomly chooses 20 images from a dataset and classifies them (proof of concept)

    import random
    from PIL import Image
    from torchvision import transforms

    model.eval()
    dataset = subset.dataset  # underlying NestedImageDataset
    class_names = list(class_to_idx.keys())  # same mapping as during training

    # Sample only from the subset indices
    indices = random.sample(subset.indices, num_images)

    for idx in indices:
        img_path, true_label = dataset.samples[idx]
        img = Image.open(img_path).convert('RGB')
        if transform:
            img_tensor = transform(img).unsqueeze(0).to(device)  # add batch dimension
        else:
            img_tensor = transforms.ToTensor()(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(img_tensor)
            pred_label = torch.argmax(output, dim=1).item()

        print(f'Image: {img_path.split("/")[-1]}')
        print(f'  True label:  {class_names[true_label]}')
        print(f'  Predicted:   {class_names[pred_label]}')
        print('-' * 40)


classify_random_images(test_dataset, model, device, num_images=50, transform=transform_eval)


Image: oral_normal_0862.jpg
  True label:  oral_normal
  Predicted:   oral_normal
----------------------------------------
Image: lymph_fl_2869.jpg
  True label:  lymph_fl
  Predicted:   lymph_fl
----------------------------------------
Image: breast_benign_1539.jpg
  True label:  breast_benign
  Predicted:   breast_benign
----------------------------------------
Image: lymph_fl_1399.jpg
  True label:  lymph_fl
  Predicted:   lymph_fl
----------------------------------------
Image: breast_malignant_1523.jpg
  True label:  breast_malignant
  Predicted:   breast_malignant
----------------------------------------
Image: brain_menin_1860.jpg
  True label:  brain_menin
  Predicted:   brain_menin
----------------------------------------
Image: oral_scc_2919.jpg
  True label:  oral_scc
  Predicted:   oral_scc
----------------------------------------
Image: brain_glioma_2804.jpg
  True label:  brain_glioma
  Predicted:   brain_glioma
----------------------------------------
Image: lymph_cll_25

In [ ]:
#------------
#FRONT END
#------------

In [ ]:
#cancer model.py
import json
from pathlib import Path

import torch
from torch import nn
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from PIL import Image


BASE_DIR = Path(file).resolve().parent


WEIGHTS_PATH = BASE_DIR / "best_model_epoch5.pt"
CLASSES_PATH = BASE_DIR / "classes.json"


with open(CLASSES_PATH, "r") as f:
    CLASS_NAMES = json.load(f)


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def load_model():

    model = resnet50(weights=ResNet50_Weights.DEFAULT)
    model.fc = nn.Linear(model.fc.in_features, 26)


    state_dict = torch.load(WEIGHTS_PATH, map_location=DEVICE)
    model.load_state_dict(state_dict)

    model.to(DEVICE)
    model.eval()
    return model



MODEL = load_model()


EVAL_TRANSFORM = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


@torch.no_grad()
def predict(image: Image.Image):
    # Ensure 3-channel RGB
    if image.mode != "RGB":
        image = image.convert("RGB")

    # Preprocess and move to device
    tensor = EVAL_TRANSFORM(image).unsqueeze(0).to(DEVICE)

    # Forward pass
    logits = MODEL(tensor)
    probs = torch.softmax(logits, dim=1)[0]

    # Top-1 prediction
    pred_idx = int(torch.argmax(probs).item())
    pred_class = CLASS_NAMES[pred_idx]

    # Full probability map
    probs_dict = {
        CLASS_NAMES[i]: float(probs[i].item())
        for i in range(len(CLASS_NAMES))
    }

    return pred_class, probs_dict

In [ ]:
#app.py
print("app.py importing...")
from flask import Flask, request
from PIL import Image
import io

from cancer_model import predict

app = Flask(name)

@app.route("/", methods=["GET"])
def index():
    return """
<!DOCTYPE html>
<html>
<head>
  <title>Cancer Classifier</title>
</head>
<body>
  <h1>Cancer Classifier</h1>
  <h2>Cancer Image Classifier</h2>

  <form action="/predict" method="post" enctype="multipart/form-data">
    <p>Select an image:</p>
    <input type="file" name="image" accept="image/*" required>
    <br><br>
    <button type="submit">Upload and Classify</button>
  </form>
</body>
</html>
"""


@app.route("/predict", methods=["POST"])
def predict_route():
    file = request.files.get("image")
    if file is None or file.filename == "":
        return "No file selected", 400

    try:
        contents = file.read()
        image = Image.open(io.BytesIO(contents))

        class_name, probs = predict(image)

        # Sort probs descending for display
        sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)

        # Build HTML result page
        html = []
        html.append("<h1>Prediction Result</h1>")
        html.append(f"<h2>Top prediction: {class_name}</h2>")

        html.append("<h3>All subclass probabilities</h3>")
        html.append("<ul>")
        for cls, p in sorted_probs:
            html.append(f"<li>{cls}: {p:.4f}</li>")
        html.append("</ul>")

        html.append('<p><a href="/">Back</a></p>')

        return "\n".join(html)
    except Exception as e:
        return f"Error: {e}", 500


if name == "main":
    app.run(debug=True)